In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import joblib

2024-04-11 08:30:54.194432: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-11 08:30:54.194830: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 08:30:54.197671: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-11 08:30:54.229974: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-11 08:30:54.817825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
df = pd.read_csv('Spam Email Detection - spam.csv', usecols=['v1', 'v2'], encoding='utf-8')
df.columns = ['Target', 'Email']
df

,Target,Email
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
df.loc[2]['Email']

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [4]:
df['Target'].value_counts()

Target
ham     4825
spam     747
Name: count, dtype: int64

In [5]:
df.isna().sum()

Target    0
Email     0
dtype: int64

In [6]:
X = df['Email']
y = df['Target'].map({'ham':0, 'spam':1})

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [9]:
VOCAB_SIZE = 200
encoder = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(X)

In [10]:
model = keras.Sequential([
    encoder,
    keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [11]:
model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [12]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_val, y_val),
                    batch_size=32)

Epoch 1/10


2024-04-11 08:30:58.737898: E tensorflow/core/util/util.cc:131] oneDNN supports DT_BOOL only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


112/112 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.7615 - loss: 0.6586 - val_accuracy: 0.8655 - val_loss: 0.4951
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8759 - loss: 0.4562 - val_accuracy: 0.8655 - val_loss: 0.3585
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.8770 - loss: 0.3111 - val_accuracy: 0.9013 - val_loss: 0.2772
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9205 - loss: 0.2252 - val_accuracy: 0.9260 - val_loss: 0.2033
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.9526 - loss: 0.1375 - val_accuracy: 0.9552 - val_loss: 0.1468
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.9664 - loss: 0.1105 - val_accuracy: 0.9574 - val_loss: 0.1320
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9689 - loss: 0.1070 - val_accuracy: 0.9630 - val_loss: 0.1208
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.9740 - loss: 0.0875 - val_accuracy: 0.961

In [13]:
model.evaluate(X_test, y_test)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9721 - loss: 0.0977


[0.0948205441236496, 0.9739910364151001]

In [14]:
# Example text for prediction
text_to_predict = df.loc[2]['Email']

# Preprocess the text using the encoder
encoded_text = pd.Series(text_to_predict)

# Predict using the trained model
predicted_logits = model.predict(encoded_text)
predicted_logits

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step


array([[0.99438524]], dtype=float32)

In [15]:
predicted_probability_spam = predicted_logits[0][0]
print(f"The predicted probability of being spam: {predicted_probability_spam}")

# Convert probabilities to class labels
predicted_class = "spam" if predicted_probability_spam > 0.5 else "ham"

# Print the prediction
print(f"The predicted class for the text '{text_to_predict}' is: {predicted_class}")


The predicted probability of being spam: 0.9943852424621582
The predicted class for the text 'Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's' is: spam
